<a href="https://colab.research.google.com/github/drew-walkerr/DetectingBias/blob/main/detecting_bias_neural_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial Links from class
1. https://www.kaggle.com/deshwalmahesh/nlp-beginner-1-rnn-lstm-gru-embeddings-glove
2. https://www.kaggle.com/kredy10/simple-lstm-for-text-classification 

3. https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/


In [13]:
!pip install keras

ERROR: Could not find a version that satisfies the requirement keras.optimizers (from versions: none)
ERROR: No matching distribution found for keras.optimizers


In [16]:
# This is a BoW classifier to detect the presence of provider scare quotes in provider clinical notes

import pandas as pd
import numpy
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import feature_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import seaborn as sn
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import MultinomialNB
# Import modules for evaluation purposes
# Import libraries for predcton
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import random # to perform randomisation of tasks
from tqdm.notebook import tqdm # for a continuous progress bar style
import time # time module 
import os # import operating system

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Load in data
gold_standard = pd.read_csv("gold_standard_bias_annotation_doc_training.csv")


Mini HW 9/10—Neural classifier —Instructions:
Due 10 Dec
In this two part HW, you will split your dataset into training/test sets, and then build a neu-
ral classifier of your choice. Recommend LSTM but for an extra challenge you can also consider a transformer/RoBERTA if you prefer (esp if you are working with a low resource language). Do this homework in python only. You will need to move to google co-lab if you haven’t already and take
advantage of a GPU.

1. First, for HW 8, set up your neural model. You will likely want to use the same training/test split
as before, but if you make any adjustments make sure to note the reason. You will need to include
Glove embeddings as a feature of your LSTM model. Consider the many available tutorials such
as this or this. Consider at least two specifications of your neural model, and write up what you
did and why.





In [26]:
# Split
seed = 1234
X = gold_standard['Sentence']
y = gold_standard['quote_use']
#Hyperparameters
max_features_model = 3000
splits = 5
model_name = "RNN"
#Splitting and model
skf = StratifiedKFold(n_splits=splits, random_state=seed, shuffle= True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

max_words = 1000
max_len = 150



In [27]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [28]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [30]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
3/3 [==============================] - 5s 559ms/step - loss: 0.6517 - accuracy: 0.2299 - val_loss: 0.4679 - val_accuracy: 0.2660
Epoch 2/10
3/3 [==============================] - 1s 229ms/step - loss: -0.0099 - accuracy: 0.1658 - val_loss: -2.5021 - val_accuracy: 0.2660


In [31]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [32]:
accr = model.evaluate(test_sequences_matrix,y_test)


4/4 [==============================] - 0s 25ms/step - loss: -1.8456 - accuracy: 0.1880


In [33]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: -1.846
  Accuracy: 0.188


2. Next, for HW 10, estimate the F1 score and plot a precision/recall curve to compare your neural
model specifications to your BOW model in the same plot. Consider using a tutorial (e.g. here).
Write up in overleaf what you did and why, and include the plot that compares your neural and
BOW models in your write up on overleaf and submit on Canvas.
